In [1]:
import pandas as pd
import numpy as np
import scipy.stats as scipy
from __future__ import division

In [2]:
df = pd.read_csv("./Data/Spain/SP1_13.csv")
df_14 = pd.read_csv("./Data/Spain/SP1_14.csv")

In [3]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'HS', u'AS', u'HST', u'AST', u'HF', u'AF',
       u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H', u'B365D', u'B365A',
       u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA', u'LBH', u'LBD', u'LBA',
       u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA', u'SJH', u'SJD', u'SJA',
       u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH', u'BbAvH', u'BbMxD',
       u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5', u'BbAv>2.5',
       u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH', u'BbAvAHH',
       u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

In [4]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date'],axis=1)
table_features = df.ix[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
bet_13 = df.ix[:,23:]


In [5]:
res_13.head()
feature_table = df.ix[:,:23]

In [6]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS',))

In [7]:
avg_home_scored_13 = res_13.FTHG.sum() / 380.0
avg_away_scored_13 = res_13.FTAG.sum() / 380.0
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.63157894737
Average number of goals away 1.11842105263
Average number of goals conceded at home 1.63157894737
Average number of goals conceded away 1.11842105263


In [8]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [9]:
# list(res_home)[0]

In [10]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Almeria,26,17,NaN,NaN,31,40,NaN,NaN
1,Ath Bilbao,42,24,NaN,NaN,18,21,NaN,NaN
2,Ath Madrid,49,28,NaN,NaN,10,16,NaN,NaN
3,Barcelona,64,36,NaN,NaN,15,18,NaN,NaN
4,Betis,19,17,NaN,NaN,31,47,NaN,NaN


In [11]:
table_13.HAS = (table_13.HGS / 19.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 19.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 19.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 19.0) / avg_away_conceded_13
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Almeria,26,17,0.838710,0.800000,31,40,1.458824,1.290323
1,Ath Bilbao,42,24,1.354839,1.129412,18,21,0.847059,0.677419
2,Ath Madrid,49,28,1.580645,1.317647,10,16,0.470588,0.516129
3,Barcelona,64,36,2.064516,1.694118,15,18,0.705882,0.580645
4,Betis,19,17,0.612903,0.800000,31,47,1.458824,1.516129


In [12]:
feature_table = feature_table[['HomeTeam','AwayTeam','FTR','HST','AST']]
f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

In [13]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS
0,Sociedad,Getafe,H,6,2,1.225806,0.894118,1.129412,1.161290
1,Valencia,Malaga,H,1,2,1.129032,1.129412,0.752941,0.935484
2,Valladolid,Ath Bilbao,A,2,3,0.645161,0.800000,0.847059,1.387097
3,Barcelona,Levante,H,13,1,2.064516,0.705882,1.694118,0.580645
4,Osasuna,Granada,A,5,4,0.645161,1.176471,0.564706,1.193548


In [14]:
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

In [15]:
feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)

In [16]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
0,Sociedad,Getafe,H,6,2,1.225806,0.894118,1.129412,1.161290,1
1,Valencia,Malaga,H,1,2,1.129032,1.129412,0.752941,0.935484,1
2,Valladolid,Ath Bilbao,A,2,3,0.645161,0.800000,0.847059,1.387097,-1
3,Barcelona,Levante,H,13,1,2.064516,0.705882,1.694118,0.580645,1
4,Osasuna,Granada,A,5,4,0.645161,1.176471,0.564706,1.193548,-1


In [17]:
X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]]
y_train = feature_table['Result']

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
clf = DecisionTreeClassifier()
clf2 = MultinomialNB()
clf3 = XGBClassifier()

clf3.fit(X_train,y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [20]:
y_pred = clf3.predict(X_train)
accuracy_score(y_pred,y_train)
scores = cross_val_score(clf2, X_train, y_train, cv=10)


In [21]:
print scores
print scores.mean()

[ 0.58974359  0.64102564  0.61538462  0.58974359  0.66666667  0.55263158
  0.51351351  0.54054054  0.62162162  0.55555556]
0.588642691274


## Adding Recent Performance

In [22]:
ht = res_13.loc[40].HomeTeam
at = res_13.loc[40].AwayTeam

In [23]:
feat_table = res_13.sort_index(ascending=False)
feat_table = feat_table[['HomeTeam','AwayTeam','FTR','FTHG','FTAG','HS','AS','HC','AC']]
feat_table["pastHS"] = 0.0
feat_table["pastHC"] = 0.0
feat_table["pastAS"] = 0.0
feat_table["pastAC"] = 0.0
feat_table["pastHG"] = 0.0
feat_table["pastAG"] = 0.0



In [24]:
feat_table.head()

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HS,AS,HC,AC,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG
379,Vallecano,Getafe,A,1,2,16,15,4,6,0.0,0.0,0.0,0.0,0.0,0.0
378,Valladolid,Granada,A,0,1,10,6,7,5,0.0,0.0,0.0,0.0,0.0,0.0
377,Sociedad,Villarreal,A,1,2,13,15,6,2,0.0,0.0,0.0,0.0,0.0,0.0
376,Sevilla,Elche,H,3,1,15,14,6,5,0.0,0.0,0.0,0.0,0.0,0.0
375,Osasuna,Betis,H,2,1,14,16,5,7,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
k = 3
for i in range(379,-1,-1):
    row = feat_table.loc[i]
    ht = row.HomeTeam
    at = row.AwayTeam
    ht_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == ht) | (feat_table.AwayTeam == ht)].head(k)
    at_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == at) | (feat_table.AwayTeam == at)].head(k)

    feat_table.set_value(i, 'pastHC', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HC + ht_stats[ht_stats["HomeTeam"] == ht].sum().HC)/k)
    feat_table.set_value(i, 'pastAC', (at_stats[at_stats["AwayTeam"] == at].sum().HC + at_stats[at_stats["HomeTeam"] == at].sum().HC)/k)
    feat_table.set_value(i, 'pastHS', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HS + ht_stats[ht_stats["HomeTeam"] == ht].sum().AS)/k)
    feat_table.set_value(i, 'pastAS', (at_stats[at_stats["AwayTeam"] == at].sum().HS + at_stats[at_stats["HomeTeam"] == at].sum().AS)/k)
    feat_table.set_value(i, 'pastHG', (ht_stats[ht_stats["AwayTeam"] == ht].sum().FTAG + ht_stats[ht_stats["HomeTeam"] == ht].sum().FTHG)/k)
    feat_table.set_value(i, 'pastAG', (at_stats[at_stats["AwayTeam"] == at].sum().FTAG + at_stats[at_stats["HomeTeam"] == at].sum().FTHG)/k)

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feat_table['HAS'] = f_HAS
feat_table['HDS'] = f_HDS
feat_table['AAS'] = f_AAS
feat_table['ADS'] = f_ADS

/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [26]:
test_table = feat_table.drop(['FTHG','FTAG','HS','AS','HC','AC'],axis=1)

In [27]:
test_table["Result"] = test_table.apply(lambda row: transformResult(row),axis=1)
test_table.sort_index(inplace=True)

In [28]:
X_train = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[0:369]
y_train = test_table['Result'].loc[0:369]
X_test = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[369:]
y_test = test_table['Result'].loc[369:]

In [29]:
X_train.head()

,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG,HAS,HDS,AAS,ADS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.903226,1.694118,0.847059,1.419355
1,0.0,0.0,0.0,0.0,0.0,0.0,0.645161,0.800000,0.847059,1.387097
2,0.0,0.0,0.0,0.0,0.0,0.0,1.225806,0.894118,1.129412,1.161290
3,0.0,0.0,0.0,0.0,0.0,0.0,1.290323,0.988235,1.364706,1.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.645161,1.176471,0.564706,1.193548


In [30]:
clf1 = DecisionTreeClassifier()
clf2 = MultinomialNB(alpha=3)
clf3 = XGBClassifier(n)
clf4 = RandomForestClassifier()
clf5 = KNeighborsClassifier(n_neighbors=5)
clf6 = LogisticRegression(solver='lbfgs',multi_class='multinomial')
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf5.fit(X_train,y_train)
clf6.fit(X_train,y_train)

NameError: name 'n' is not defined

In [ ]:
for c in [0.1,0.5,1,1.5,2,5,10]:
    clf6 = LogisticRegression(C=c,solver='lbfgs',multi_class='multinomial')
    clf6.fit(X_train,y_train)
    scores = accuracy_score(y_test,clf6.predict(X_test))
    print scores, scores.mean()

In [ ]:
for b in xrange(1,50):
    clf5 = KNeighborsClassifier(n_neighbors=b)
    clf5.fit(X_train,y_train)
    scores = accuracy_score(y_test,clf5.predict(X_test))
    scores, scores.mean()
    #Best n =16 to 22

# Adding home advantage

In [ ]:
test_table["pastCornerDiff"] = test_table["pastHC"] - test_table["pastAC"]
test_table["pastGoalDiff"] = test_table["pastHG"] - test_table["pastAG"]
test_table["pastShotsDiff"] = test_table["pastHS"] - test_table["pastAG"]

In [ ]:
test_table.drop(['pastHC','pastAS','pastAC','pastHG','pastAG'],axis=1)
X_train = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[0:369]
y_train = test_table['Result'].loc[0:369]
X_test = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[369:]
y_test = test_table['Result'].loc[369:]



In [ ]:
clf1.fit(X_train,y_train)
#clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf5.fit(X_train,y_train)
clf6.fit(X_train,y_train)

In [ ]:
accuracy_score(y_test,clf6.predict(X_test))

In [ ]:
 from sklearn import preprocessing

In [ ]:
X_normalized = preprocessing.normalize(X_train, norm='l2')
X_test_norm = preprocessing.normalize(X_test, norm='l2')

In [ ]:
clf1.fit(X_normalized,y_train)
#clf2.fit(X_normalized,y_train)
clf3.fit(X_normalized,y_train)
clf4.fit(X_normalized,y_train)
clf5.fit(X_normalized,y_train)
clf6.fit(X_normalized,y_train)

In [ ]:
accuracy_score(y_test,clf5.predict(X_test_norm))